In [59]:
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense,Conv2D,Flatten,Dropout,MaxPooling2D,LeakyReLU,BatchNormalization,GlobalAveragePooling2D,GlobalMaxPooling2D,AveragePooling2D
from keras import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, initializers
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.applications import DenseNet121

In [60]:
early_stop = EarlyStopping(
    monitor='val_loss',      
    patience=5,              
    restore_best_weights=True
)

In [61]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/kaggle/input/aiderdata/AIDER",
    validation_split=0.15,
    subset="training",
    seed=123,
    image_size=(224, 224),
    batch_size=64
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/kaggle/input/aiderdata/AIDER",
    validation_split=0.15,
    subset="validation",
    seed=123,
    image_size=(224, 224),
    batch_size=64
)

Found 6433 files belonging to 5 classes.
Using 5469 files for training.
Found 6433 files belonging to 5 classes.
Using 964 files for validation.


In [62]:
from tensorflow.keras import layers
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),         
    layers.RandomRotation(0.25),             
    layers.RandomTranslation(0.1, 0.1),
    layers.RandomContrast(0.1)
])

In [63]:
normalization_layer = layers.Rescaling(1./255)
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.map(lambda x, y: (data_augmentation(normalization_layer(x)), y), num_parallel_calls=AUTOTUNE)
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y), num_parallel_calls=AUTOTUNE)
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

In [64]:
weights_Add="/kaggle/input/resnet-wts/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"
base_model = ResNet50(weights=weights_Add, include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

In [65]:
model = Sequential([
    base_model,
    Flatten(),
    Dense(64),
    LeakyReLU(alpha=0.1),
    Dense(5, activation='softmax')  
])

In [66]:
model.summary()

Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 100352)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 64)             │     6,422,592 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_25 (LeakyReLU)      │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 30,010,629 (114.48 MB)

 Trainable params: 6,422,917 (24.50 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [67]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
history=model.fit(train_ds,epochs=30,validation_data=val_ds,callbacks=[early_stop])

Epoch 1/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 62s 507ms/step - accuracy: 0.5514 - loss: 6.4138 - val_accuracy: 0.6919 - val_loss: 0.9334
Epoch 2/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 32s 357ms/step - accuracy: 0.6540 - loss: 1.0533 - val_accuracy: 0.7127 - val_loss: 0.9103
Epoch 3/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 31s 351ms/step - accuracy: 0.6660 - loss: 1.0194 - val_accuracy: 0.6940 - val_loss: 0.9375
Epoch 4/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 32s 364ms/step - accuracy: 0.6826 - loss: 0.9695 - val_accuracy: 0.7064 - val_loss: 0.9560
Epoch 5/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 32s 362ms/step - accuracy: 0.6628 - loss: 1.0767 - val_accuracy: 0.6992 - val_loss: 0.8583
Epoch 6/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 32s 360ms/step - accuracy: 0.6833 - loss: 0.9739 - val_accuracy: 0.7075 - val_loss: 0.9165
Epoch 7/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 33s 367ms/step - accuracy: 0.6753 - loss: 0.9460 - val_accuracy: 0.7241 - val_loss: 0.8273
Epoch 8/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 32s 364ms/step - accuracy: 0.6819 - loss: 0.9465 - val_accu